# Read in data

In [153]:
from datetime import timedelta
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

airports = [
    "KATL",
    "KCLT",
    "KDEN",
    "KDFW",
    "KJFK",
    "KMEM",
    "KMIA",
    "KORD",
    "KPHX",
    "KSEA",
]

In [181]:
DATA_DIRECTORY = Path("code execution development data")

## LAMP

In [182]:
def read_lamp(airport):
    lamp = pd.read_csv(
        DATA_DIRECTORY / airport / f"{airport}_lamp.csv.bz2",
        parse_dates=["timestamp", "forecast_timestamp"],
        dtype={"temperature": "int16", "wind_direction":"int16", "wind_gust":"int16", "cloud_ceiling":"float16", "visibility":"int16"}
    )
    return lamp

## Config

In [183]:
def read_config(airport):
    config = pd.read_csv(
        DATA_DIRECTORY / airport / f"{airport}_config.csv.bz2",
        parse_dates=["timestamp", "start_time"]
    )
    config.departure_runways = config.departure_runways.astype(str)
    return config

## Runways

In [184]:
def read_runways(airport):
    runways = pd.read_csv(
        DATA_DIRECTORY / airport / f"{airport}_runways.csv.bz2",
        parse_dates=["timestamp", "departure_runway_actual_time"]
    )
    return runways

## ETD

In [185]:
def read_etd(airport):
    etd = pd.read_csv(
        DATA_DIRECTORY / airport / f"{airport}_etd.csv.bz2",
        parse_dates=["departure_runway_estimated_time", "timestamp"]
    )
    return etd

## MFS

In [186]:
def read_mfs(airport):
    etd = pd.read_csv(
        DATA_DIRECTORY / airport / f"{airport}_mfs.csv.bz2"
    )
    return etd

# Initial Model, Linear Regression

In [196]:
submission_format = pd.read_csv(
    "code execution development data/test_labels.csv", parse_dates=["timestamp"]
)

## Insert Data

In [205]:
def filter_lamp(current_time):
    # weather data
    valid_lamp = lamp.loc[(lamp.timestamp <= current_time) 
                    & (lamp.timestamp > valid_time) 
                    & (lamp.forecast_timestamp <= current_time) 
                    & (lamp.forecast_timestamp > valid_time)]
    return valid_lamp.iloc[-1]

def filter_departures(current_time):
    # list of active runways
    valid_departures = config.loc[(config.timestamp <= current_time) 
                    & (config.timestamp > valid_time) 
                    & (config.start_time <= current_time) 
                    & (config.start_time > crunch_time)]
    try:
        valid_departures = valid_departures.iloc[-1].departure_runways
        return valid_departures
    except:
        return None

def filter_dep_runways(current_time):
    # runway to be used
    dep_runways = runways.loc[(runways.timestamp <= current_time) 
                & (runways.timestamp > valid_time) 
                & (runways.departure_runway_actual_time <= current_time) 
                & (runways.departure_runway_actual_time > crunch_time)]
    return dep_runways['gufi'].nunique()

def filter_arr_runways(current_time):
    # runway to be used
    arr_runways = runways.loc[(runways.timestamp <= current_time) 
                & (runways.timestamp > valid_time) 
                & (runways.departure_runway_actual_time <= current_time) 
                & (runways.departure_runway_actual_time > crunch_time)]
    return arr_runways['gufi'].nunique()
    
def filter_etd(current_time):    
    #etd data
    valid_etd = etd.loc[(etd.timestamp <= current_time) 
                    & (etd.timestamp > valid_time) 
                    & (etd.gufi == df.loc[i].gufi)]
    try:
        return (valid_etd.iloc[-1].departure_runway_estimated_time - current_time).total_seconds()
    except:
        return None
    
def weekday(current_time):
    weekday = current_time.weekday()
    values = {0:"Monday",
              1:"Tuesday",
              2:"Wednesday",
              3:"Thursday",
              4:"Friday",
              5:"Saturday",
              6:"Sunday",}
    return values[current_time.weekday()]

In [202]:
import warnings
warnings.filterwarnings('ignore')

## KDEN

In [203]:
submission_format = pd.read_csv(
    "prescreened train labels/prescreened_train_labels_KDEN.csv.bz2", parse_dates=["timestamp"]
)
submission_format = submission_format.iloc[0:1000]
submission_format.shape

(1000, 4)

In [204]:
full_frame = pd.DataFrame()
airport = 'KDEN'
df = submission_format[submission_format.airport == airport]
times = df.timestamp.unique()

etd = read_etd(airport)
config = read_config(airport)
runways = read_runways(airport)
lamp = read_lamp(airport)

for t in times:
    indices = df[df.timestamp == t].index
    current_time = pd.to_datetime(t)
    valid_time = current_time - pd.Timedelta(30, unit='hours')
    crunch_time = current_time - pd.Timedelta(1, unit='hours')

    # insert day of week
    df.loc[indices, 'weekday'] = weekday(current_time)

    # insert etd data
    for i in indices:
        df.loc[i, 'etd'] = filter_etd(current_time)

    # insert traffic data
    df.loc[indices, 'departures'] = filter_departures(current_time)
    df.loc[indices, 'arrivals'] = filter_arrivals(current_time)
    df.loc[indices, 'dep_traffic'] = filter_dep_runways(current_time)

    # insert weather data
    current_forecast = filter_lamp(current_time)
    df.loc[indices, 'precip'] = current_forecast.precip
    df.loc[indices, 'lightning_prob'] = current_forecast.lightning_prob
    df.loc[indices, 'cloud'] = current_forecast.cloud
    df.loc[indices, 'visibility'] = current_forecast.visibility
    df.loc[indices, 'cloud_ceiling'] = current_forecast.cloud_ceiling
    df.loc[indices, 'wind_gust'] = current_forecast.wind_gust
    df.loc[indices, 'wind_speed'] = current_forecast.wind_speed
    df.loc[indices, 'wind_direction'] = current_forecast.wind_direction
    df.loc[indices, 'temperature'] = current_forecast.temperature

full_frame = pd.concat([full_frame, df])
        
# insert metadata
full_frame = full_frame.merge(read_mfs('KDEN'), how='left', on='gufi')

full_frame.head(5)

2022-04-05 14:00:00
2022-04-05 14:15:00
2022-04-05 14:30:00
2022-04-05 14:45:00
2022-04-05 15:00:00
2022-04-06 14:00:00
2022-04-06 14:15:00
2022-04-06 14:30:00
2022-04-06 14:45:00
2022-04-06 15:00:00
2022-04-07 14:00:00
2022-04-07 14:15:00
2022-04-07 14:30:00
2022-04-07 14:45:00
2022-04-07 15:00:00
2022-04-08 14:15:00
2022-04-08 14:30:00
2022-04-08 14:45:00
2022-04-08 15:00:00
2022-04-08 15:15:00
2022-04-08 15:30:00
2022-04-09 14:00:00
2022-04-09 14:15:00
2022-04-09 14:30:00
2022-04-09 14:45:00
2022-04-09 15:00:00
2022-04-09 15:15:00
2022-04-09 15:30:00
2022-04-09 15:45:00
2022-04-09 16:00:00
2022-04-09 16:15:00
2022-04-09 16:30:00
2022-04-09 16:45:00
2022-04-09 17:00:00
2022-04-09 17:15:00
2022-04-09 17:30:00
2022-04-10 14:00:00
2022-04-10 14:15:00
2022-04-10 14:30:00
2022-04-10 14:45:00
2022-04-10 15:00:00
2022-04-11 14:15:00
2022-04-11 14:30:00
2022-04-11 14:45:00
2022-04-11 15:00:00
2022-04-12 14:00:00
2022-04-12 14:15:00
2022-04-12 14:30:00
2022-04-12 14:45:00
2022-04-12 15:00:00


IndexError: single positional indexer is out-of-bounds

## All Airports

In [144]:
full_frame = pd.DataFrame()
for airport in airports:
    df = submission_format[submission_format.airport == airport]
    times = df.timestamp.unique()

    etd = read_etd(airport)
    config = read_config(airport)
    runways = read_runways(airport)
    lamp = read_lamp(airport)

    for t in times:
        indices = df[df.timestamp == t].index
        current_time = pd.to_datetime(t)
        valid_time = current_time - pd.Timedelta(30, unit='hours')
        crunch_time = current_time - pd.Timedelta(1, unit='hours')
        
        # insert day of week
        df.loc[indices, 'weekday'] = weekday(current_time)
        
        # insert etd data
        for i in indices:
            df.loc[i, 'etd'] = filter_etd(current_time)
            
        # insert traffic data
        df.loc[indices, 'departures'] = filter_departures(current_time)
        df.loc[indices, 'arrivals'] = filter_arrivals(current_time)
        df.loc[indices, 'dep_traffic'] = filter_dep_runways(current_time)

        # insert weather data
        current_forecast = filter_lamp(current_time)
        df.loc[indices, 'precip'] = current_forecast.precip
        df.loc[indices, 'lightning_prob'] = current_forecast.lightning_prob
        df.loc[indices, 'cloud'] = filter_lamp(current_time).cloud
        df.loc[indices, 'visibility'] = current_forecast.visibility
        df.loc[indices, 'cloud_ceiling'] = current_forecast.cloud_ceiling
        df.loc[indices, 'wind_gust'] = current_forecast.wind_gust
        df.loc[indices, 'wind_speed'] = current_forecast.wind_speed
        df.loc[indices, 'wind_direction'] = current_forecast.wind_direction
        df.loc[indices, 'temperature'] = current_forecast.temperature
        
    full_frame = pd.concat([full_frame, df])
        
# insert metadata
metadata = pd.concat([read_mfs('KATL'),
                      read_mfs('KCLT'),
                      read_mfs('KDEN'),
                      read_mfs('KDFW'),
                      read_mfs('KJFK'),
                      read_mfs('KMEM'),
                      read_mfs('KMIA'),
                      read_mfs('KORD'),
                      read_mfs('KPHX'),
                      read_mfs('KSEA')])

full_frame = full_frame.merge(metadata, how='left', on='gufi')

full_frame.head(5)

,gufi,timestamp,airport,minutes_until_pushback,weekday,etd,departures,arrivals,dep_traffic,precip,...,cloud_ceiling,wind_gust,wind_speed,wind_direction,temperature,aircraft_engine_class,aircraft_type,major_carrier,flight_type,isdeparture
0,AAL1227.ATL.MIA.201114.1242.0052.TFM,2020-11-15 11:15:00,KATL,86,Sunday,6300.0,"8R, 9L","10, 8L, 9R",1.0,False,...,8.0,0.0,2.0,13.0,54.0,JET,A319,AAL,SCHEDULED_AIR_TRANSPORT,True
1,AAL1227.ATL.MIA.201114.1242.0052.TFM,2020-11-15 11:15:00,KATL,86,Sunday,6300.0,"8R, 9L","10, 8L, 9R",1.0,False,...,8.0,0.0,2.0,13.0,54.0,JET,A319,AAL,SCHEDULED_AIR_TRANSPORT,False
2,AAL1227.ATL.MIA.201114.1242.0052.TFM,2020-11-15 12:00:00,KATL,41,Sunday,3600.0,"8R, 9L","10, 8L",4.0,False,...,8.0,0.0,5.0,14.0,53.0,JET,A319,AAL,SCHEDULED_AIR_TRANSPORT,True
3,AAL1227.ATL.MIA.201114.1242.0052.TFM,2020-11-15 12:00:00,KATL,41,Sunday,3600.0,"8R, 9L","10, 8L",4.0,False,...,8.0,0.0,5.0,14.0,53.0,JET,A319,AAL,SCHEDULED_AIR_TRANSPORT,False
4,AAL153.ATL.CLT.201114.1137.0016.TFM,2020-11-15 10:30:00,KATL,62,Sunday,4980.0,"8R, 9L","10, 8L, 9R",0.0,False,...,8.0,0.0,5.0,11.0,53.0,JET,A320,AAL,SCHEDULED_AIR_TRANSPORT,True


# Data Prep

## Distribution

In [145]:
full_frame.select_dtypes(include='number').describe()

,minutes_until_pushback,etd,dep_traffic,visibility,cloud_ceiling,wind_gust,wind_speed,wind_direction,temperature
count,2356.000000,2356.000000,2356.000000,2356.000000,2356.000000,2356.000000,2356.000000,2356.000000,2356.000000
mean,44.109932,3453.388795,28.724958,6.750849,7.180390,6.518251,10.754669,18.072581,56.983871
std,28.722366,1638.195583,23.640350,0.773979,1.421451,11.394941,6.077706,7.727816,15.545719
min,1.000000,-10920.000000,0.000000,1.000000,2.000000,0.000000,1.000000,1.000000,22.000000
25%,21.000000,2280.000000,10.000000,7.000000,7.000000,0.000000,5.000000,12.000000,46.000000
50%,43.000000,3540.000000,22.000000,7.000000,8.000000,0.000000,9.000000,16.000000,58.000000
75%,63.000000,4740.000000,44.000000,7.000000,8.000000,19.000000,16.000000,24.000000,75.000000
max,271.000000,9840.000000,85.000000,7.000000,8.000000,31.000000,23.000000,35.000000,80.000000


In [146]:
full_frame.select_dtypes(exclude='number').describe()

,gufi,timestamp,airport,weekday,departures,arrivals,precip,lightning_prob,cloud,aircraft_engine_class,aircraft_type,major_carrier,flight_type,isdeparture
count,2356,2356,2356,2356,2132,2132,2356,2356,2356,2356,2356,1902,2182,2356
unique,1228,17,10,1,22,24,2,3,5,2,37,3,2,2
top,DAL8903.MIA.ATL.201114.0057.0044.TFM,2020-11-15 00:00:00,KATL,Sunday,"8R, 9L","13R, 17C, 17L, 18L",False,N,FW,JET,A321,AAL,SCHEDULED_AIR_TRANSPORT,True
freq,8,485,416,2356,416,355,1991,2338,681,2320,322,958,2170,2046
first,NaN,2020-11-15 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,2020-11-15 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Missing Values

In [122]:
full_frame.isnull().sum()

gufi                        0
timestamp                   0
airport                     0
minutes_until_pushback      0
etd                         0
departures                182
arrivals                  182
dep_traffic                 0
dep_actual                  0
precip                      0
lightning_prob              0
cloud                       0
visibility                  0
cloud_ceiling               0
wind_gust                   0
wind_speed                  0
wind_direction              0
temperature                 0
aircraft_engine_class       0
aircraft_type               0
major_carrier             454
flight_type               174
isdeparture                 0
dtype: int64

In [123]:
# impute missing values with mode
full_frame.major_carrier.fillna(full_frame.major_carrier.mode()[0], inplace=True)
full_frame.flight_type.fillna(full_frame.flight_type.mode()[0], inplace=True)

## Outliers

In [124]:
from scipy import stats
z_scores = stats.zscore(full_frame.select_dtypes(include='number').describe())
z_scores

,minutes_until_pushback,etd,dep_traffic,dep_actual,visibility,cloud_ceiling,wind_gust,wind_speed,wind_direction,temperature
count,2.631246,0.043772,2.642264,2.645128,2.645737,2.645737,2.645522,2.645661,2.645547,2.644721
mean,-0.406501,0.243871,-0.379597,-0.380913,-0.375991,-0.376371,-0.382034,-0.377132,-0.375589,-0.369322
std,-0.426719,-0.087114,-0.402770,-0.380371,-0.383678,-0.383780,-0.375750,-0.383160,-0.388957,-0.423648
min,-0.463146,-2.376999,-0.452096,-0.399774,-0.383388,-0.383036,-0.390434,-0.389705,-0.397651,-0.415187
25%,-0.436866,0.029914,-0.416635,-0.397189,-0.375670,-0.376603,-0.390434,-0.384549,-0.383437,-0.383722
50%,-0.407959,0.259664,-0.387740,-0.386850,-0.375670,-0.375316,-0.390434,-0.379394,-0.378268,-0.367990
75%,-0.381680,0.478474,-0.331264,-0.375218,-0.375670,-0.375316,-0.365950,-0.370371,-0.367930,-0.345703
max,-0.108375,1.408418,-0.272161,-0.324814,-0.375670,-0.375316,-0.350487,-0.361349,-0.353715,-0.339148


In [125]:
z_scores[z_scores.abs() >= 3].count()

minutes_until_pushback    0
etd                       0
dep_traffic               0
dep_actual                0
visibility                0
cloud_ceiling             0
wind_gust                 0
wind_speed                0
wind_direction            0
temperature               0
dtype: int64

## Feature Engineering

In [126]:
full_frame.nunique()

gufi                      1228
timestamp                   17
airport                     10
minutes_until_pushback     133
etd                        142
departures                  22
arrivals                    24
dep_traffic                 62
dep_actual                  31
precip                       2
lightning_prob               3
cloud                        5
visibility                   4
cloud_ceiling                6
wind_gust                   13
wind_speed                  23
wind_direction              29
temperature                 39
aircraft_engine_class        2
aircraft_type               37
major_carrier                3
flight_type                  2
isdeparture                  2
dtype: int64

In [127]:
# binary encoding
full_frame.replace(False, 0, inplace=True)
full_frame.replace(True, 0, inplace=True)

In [128]:
# nominal encoding
full_frame = pd.get_dummies(full_frame, columns=['airport', 
                                                 'departures', 
                                                 'arrivals',
                                                 'cloud_ceiling', 
                                                 'visibility',
                                                 'cloud', 
                                                 'lightning_prob',
                                                 'aircraft_engine_class',
                                                 'aircraft_type',
                                                 'major_carrier',
                                                 'flight_type'], 
                    drop_first=True)
full_frame.head(5)

,gufi,timestamp,minutes_until_pushback,etd,dep_traffic,dep_actual,precip,wind_gust,wind_speed,wind_direction,...,aircraft_type_E170,aircraft_type_E55P,aircraft_type_E75L,aircraft_type_E75S,aircraft_type_F900,aircraft_type_MD11,aircraft_type_PC12,major_carrier_DAL,major_carrier_UAL,flight_type_SCHEDULED_AIR_TRANSPORT
0,AAL1227.ATL.MIA.201114.1242.0052.TFM,2020-11-15 11:15:00,86,6300.0,2.0,0.0,0,0.0,2.0,13.0,...,0,0,0,0,0,0,0,0,0,1
1,AAL1227.ATL.MIA.201114.1242.0052.TFM,2020-11-15 11:15:00,86,6300.0,2.0,0.0,0,0.0,2.0,13.0,...,0,0,0,0,0,0,0,0,0,1
2,AAL1227.ATL.MIA.201114.1242.0052.TFM,2020-11-15 12:00:00,41,3600.0,4.0,2.0,0,0.0,5.0,14.0,...,0,0,0,0,0,0,0,0,0,1
3,AAL1227.ATL.MIA.201114.1242.0052.TFM,2020-11-15 12:00:00,41,3600.0,4.0,2.0,0,0.0,5.0,14.0,...,0,0,0,0,0,0,0,0,0,1
4,AAL153.ATL.CLT.201114.1137.0016.TFM,2020-11-15 10:30:00,62,4980.0,2.0,0.0,0,0.0,5.0,11.0,...,0,0,0,0,0,0,0,0,0,1


# Linear Regression

## Model 1: All Features

In [129]:
import statsmodels.api as sm

Y = full_frame['minutes_until_pushback']
X = full_frame.drop(columns=['gufi', 'timestamp', 'minutes_until_pushback'])
X = sm.add_constant(X)

model = sm.OLS(Y,X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                              
==================================================================================
Dep. Variable:     minutes_until_pushback   R-squared:                       0.593
Model:                                OLS   Adj. R-squared:                  0.578
Method:                     Least Squares   F-statistic:                     38.99
Date:                    Sun, 02 Apr 2023   Prob (F-statistic):               0.00
Time:                            13:55:47   Log-Likelihood:                -10193.
No. Observations:                    2356   AIC:                         2.056e+04
Df Residuals:                        2270   BIC:                         2.105e+04
Df Model:                              85                                         
Covariance Type:                nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
const                                  -9.9556      9.233     -1.078      0.281     -28.062       8.150
etd                                     0.0132      0.000     52.063      0.000       0.013       0.014
dep_traffic                             0.0634      0.032      2.000      0.046       0.001       0.125
dep_actual                             -0.1388      0.057     -2.442      0.015      -0.250      -0.027
precip                               7.742e-14   1.66e-13      0.466      0.641   -2.48e-13    4.03e-13
wind_gust                               0.1346      0.332      0.405      0.686      -0.517       0.786
wind_speed                             -0.2857      0.399     -0.716      0.474      -1.069       0.497
wind_direction                         -0.5542      0.247     -2.241      0.025      -1.039      -0.069
temperature                             0.9316      0.252      3.695      0.000       0.437       1.426
isdeparture                          -5.55e-14   6.44e-14     -0.862      0.389   -1.82e-13    7.07e-14
airport_KCLT                            7.0372      4.508      1.561      0.119      -1.803      15.878
airport_KDEN                            7.1859      5.086      1.413      0.158      -2.788      17.159
airport_KDFW                           -8.1298      3.282     -2.477      0.013     -14.566      -1.693
airport_KJFK                           -1.1721      2.272     -0.516      0.606      -5.627       3.283
airport_KMEM                           12.9936      4.293      3.027      0.003       4.575      21.413
airport_KMIA                          -12.1224      3.055     -3.968      0.000     -18.113      -6.132
airport_KORD                           12.3859      5.084      2.436      0.015       2.417      22.355
airport_KPHX                          -28.3611      5.637     -5.031      0.000     -39.415     -17.307
airport_KSEA                           13.2342      3.625      3.651      0.000       6.125      20.343
departures_16L                         13.2342      3.625      3.651      0.000       6.125      20.343
departures_17R                          4.2051      2.786      1.509      0.131      -1.259       9.669
departures_17R, 18L                   -10.1994      2.775     -3.675      0.000     -15.641      -4.757
departures_18C, 18L                     6.8095      6.108      1.115      0.265      -5.169      18.788
departures_18C, 18L, 18R               -5.9173      3.190     -1.855      0.064     -12.172       0.338
departures_22L, 28R                    -0.9924      7.278     -0.136      0.892     -15.264      13.279
departures_25, 34L                      1.5461      6.418      0.241      0.810     -11.039      14.131
departures_27, 36C, 36L

In [96]:
# calculate MAE
from sklearn.metrics import mean_absolute_error as mae
ypred = model.predict()
MAE = mae(Y, ypred)
MAE

9.964373756871446

In [48]:
# check for multicollinearity
from statsmodels.stats.outliers_influence import variance_inflation_factor

# create VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
  
# calculate VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]
  
vif_data

,feature,VIF
0,const,0.000000
1,etd,1.258801
2,runway_traffic,3.719041
3,precip,NaN
4,wind_gust,95.218744
...,...,...
111,aircraft_type_MD11,3.027002
112,aircraft_type_PC12,inf
113,major_carrier_DAL,2.798568
114,major_carrier_UAL,3.140969


## Model 2: Reduced Features

In [49]:
# reduce features
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SequentialFeatureSelector

# re-define independent variables
X = full_frame.drop(columns=['gufi', 'timestamp', 'minutes_until_pushback'])

# linear regression estimator to be used in feature selection
reg = LinearRegression().fit(X, Y)

In [50]:
# reduce features using SequentialFeatureSelector
sfs = SequentialFeatureSelector(reg, n_features_to_select=23)
sfs.fit(X, Y)

SequentialFeatureSelector(estimator=LinearRegression(), n_features_to_select=23)

In [51]:
# return selected features
sfs.get_feature_names_out()

array(['etd', 'runway_traffic', 'wind_direction', 'airport_KMIA',
       'airport_KORD', 'departures_36C, 36R', 'departures_36R',
       'departures_8L, 8R, 9', 'arrivals_10C', 'arrivals_18L, 18R, 27',
       'cloud_ceiling_4.0', 'cloud_ceiling_6.0', 'cloud_ceiling_7.0',
       'aircraft_type_A320', 'aircraft_type_A333', 'aircraft_type_B752',
       'aircraft_type_B763', 'aircraft_type_B764', 'aircraft_type_CRJ7',
       'aircraft_type_CRJ9', 'aircraft_type_E145', 'aircraft_type_E170',
       'aircraft_type_E75L'], dtype=object)

In [52]:
# re-fit model
X = full_frame[['etd', 'runway_traffic', 'wind_direction', 'airport_KMIA',
       'airport_KORD', 'departures_36C, 36R', 'departures_36R',
       'departures_8L, 8R, 9', 'arrivals_10C', 'arrivals_18L, 18R, 27',
       'cloud_ceiling_4.0', 'cloud_ceiling_6.0', 'cloud_ceiling_7.0',
       'aircraft_type_A320', 'aircraft_type_A333', 'aircraft_type_B752',
       'aircraft_type_B763', 'aircraft_type_B764', 'aircraft_type_CRJ7',
       'aircraft_type_CRJ9', 'aircraft_type_E145', 'aircraft_type_E170',
       'aircraft_type_E75L']]
X = sm.add_constant(X)

model2 = sm.OLS(Y,X).fit()
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                              
==================================================================================
Dep. Variable:     minutes_until_pushback   R-squared:                       0.564
Model:                                OLS   Adj. R-squared:                  0.560
Method:                     Least Squares   F-statistic:                     131.3
Date:                    Sun, 02 Apr 2023   Prob (F-statistic):               0.00
Time:                            13:26:28   Log-Likelihood:                -10274.
No. Observations:                    2356   AIC:                         2.060e+04
Df Residuals:                        2332   BIC:                         2.074e+04
Df Model:                              23                                         
Covariance Type:                nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     0.8761      1.649      0.531      0.595      -2.357       4.110
etd                       0.0131      0.000     53.041      0.000       0.013       0.014
runway_traffic            0.0472      0.018      2.614      0.009       0.012       0.083
wind_direction           -0.1043      0.064     -1.639      0.101      -0.229       0.021
airport_KMIA             84.0623     13.775      6.103      0.000      57.050     111.075
airport_KORD             -6.6191      5.480     -1.208      0.227     -17.365       4.127
departures_36C, 36R      -8.9439      1.891     -4.730      0.000     -12.652      -5.236
departures_36R           68.2202     19.190      3.555      0.000      30.589     105.851
departures_8L, 8R, 9    -80.9777     13.802     -5.867      0.000    -108.043     -53.912
arrivals_10C             25.2492     13.597      1.857      0.063      -1.413      51.912
arrivals_18L, 18R, 27    20.1833      4.507      4.478      0.000      11.345      29.022
cloud_ceiling_4.0        -2.0273      5.352     -0.379      0.705     -12.523       8.468
cloud_ceiling_6.0         5.2822      1.781      2.967      0.003       1.790       8.774
cloud_ceiling_7.0         1.5392      1.768      0.870      0.384      -1.929       5.007
aircraft_type_A320       -2.8755      1.757     -1.636      0.102      -6.321       0.571
aircraft_type_A333      -15.8218      9.564     -1.654      0.098     -34.576       2.933
aircraft_type_B752        7.8638      2.324      3.384      0.001       3.307      12.421
aircraft_type_B763        5.2958      2.774      1.909      0.056      -0.144      10.735
aircraft_type_B764       13.7197      4.975      2.758      0.006       3.963      23.476
aircraft_type_CRJ7       -4.7879      1.769     -2.706      0.007      -8.258      -1.318
aircraft_type_CRJ9       -4.1900      1.598     -2.622      0.009      -7.324      -1.056
aircraft_type_E145       -8.6847      2.534     -3.427      0.001     -13.654      -3.715
aircraft_type_E170       -9.5902      2.150     -4.460      0.000     -13.807      -5.373
aircraft_type_E75L       -6.8372      1.744     -3.920      0.000     -10.257      -3.417
==============================================================================
Omnibus:                     2343.044   Durbin-Watson:                   0.873
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           148218.391
Skew:                           4.743   Prob(JB):                         0.00
Kurtosis:                      40.681   Cond. No.                     1.90e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 

In [53]:
# re-calculate MAE
from sklearn.metrics import mean_absolute_error as mae
ypred = model.predict()
MAE = mae(Y, ypred)
MAE

9.96595993166621

In [54]:
# re-check for multicollinearity
from statsmodels.stats.outliers_influence import variance_inflation_factor

# create VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
  
# calculate VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]
  
vif_data

,feature,VIF
0,const,17.649588
1,etd,1.064903
2,runway_traffic,1.181050
3,wind_direction,1.571184
4,airport_KMIA,90.882628
5,airport_KORD,15.552806
6,"departures_36C, 36R",1.113235
7,departures_36R,1.014175
8,"departures_8L, 8R, 9",90.359862
9,arrivals_10C,1.017828


## Model 3: PCR

In [55]:
# perform PCA
import numpy as np
from sklearn.preprocessing import StandardScaler 
from sklearn.decomposition import PCA

data = full_frame.drop(columns=['gufi', 'timestamp', 'minutes_until_pushback'])
scale = StandardScaler()
X = scale.fit_transform(data)


X

array([[ 1.73801929, -1.17303015,  0.        , ..., -0.57212232,
        -0.42677568,  0.07155036],
       [ 1.73801929, -1.17303015,  0.        , ..., -0.57212232,
        -0.42677568,  0.07155036],
       [ 0.08951454, -1.04610154,  0.        , ..., -0.57212232,
        -0.42677568,  0.07155036],
       ...,
       [-0.4233536 , -0.1152917 ,  0.        , ..., -0.57212232,
         2.34315132,  0.07155036],
       [ 1.1885177 , -1.17303015,  0.        , ..., -0.57212232,
         2.34315132,  0.07155036],
       [ 1.1885177 , -1.17303015,  0.        , ..., -0.57212232,
         2.34315132,  0.07155036]])

In [56]:
pca=PCA()
X_red = pca.fit_transform(X)

In [57]:
np.cumsum(np.round(pca.explained_variance_ratio_, decimals = 4)*100)[0:20]

array([ 6.86, 12.84, 17.01, 20.91, 24.67, 27.84, 30.92, 33.69, 36.35,
       38.81, 41.09, 43.11, 45.07, 47.01, 48.88, 50.73, 52.54, 54.19,
       55.76, 57.31])

In [58]:
X = sm.add_constant(X)
model = sm.OLS(Y,X[:,0:20]).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                              
==================================================================================
Dep. Variable:     minutes_until_pushback   R-squared:                       0.544
Model:                                OLS   Adj. R-squared:                  0.541
Method:                     Least Squares   F-statistic:                     174.5
Date:                    Sun, 02 Apr 2023   Prob (F-statistic):               0.00
Time:                            13:26:56   Log-Likelihood:                -10328.
No. Observations:                    2356   AIC:                         2.069e+04
Df Residuals:                        2339   BIC:                         2.079e+04
Df Model:                              16                                         
Covariance Type:                nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         44.1099      0.401    110.024      0.000      43.324      44.896
x1            20.7445      0.411     50.516      0.000      19.939      21.550
x2             1.6701      0.569      2.937      0.003       0.555       2.785
x3         -8.475e-16   3.04e-16     -2.787      0.005   -1.44e-15   -2.51e-16
x4            -0.9726      1.747     -0.557      0.578      -4.399       2.453
x5            -0.0802      2.009     -0.040      0.968      -4.020       3.859
x6             0.4454      0.917      0.486      0.627      -1.353       2.244
x7            -0.7226      1.964     -0.368      0.713      -4.574       3.128
x8          2.122e-15   6.35e-16      3.341      0.001    8.76e-16    3.37e-15
x9            -2.3018      0.473     -4.867      0.000      -3.229      -1.374
x10           -1.2118      2.090     -0.580      0.562      -5.310       2.887
x11           -2.1968      1.392     -1.579      0.115      -4.926       0.532
x12           -0.4597      0.647     -0.711      0.477      -1.728       0.809
x13            2.9495      0.674      4.379      0.000       1.629       4.270
x14            1.2909      0.827      1.561      0.119      -0.331       2.913
x15           -2.8131      0.957     -2.938      0.003      -4.691      -0.936
x16           -1.5651      0.735     -2.129      0.033      -3.006      -0.124
x17            0.0962      0.456      0.211      0.833      -0.797       0.990
x18            0.0962      0.456      0.211      0.833      -0.797       0.990
x19           -0.0423      0.511     -0.083      0.934      -1.044       0.960
==============================================================================
Omnibus:                     2212.157   Durbin-Watson:                   0.867
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           105842.169
Skew:                           4.417   Prob(JB):                         0.00
Kurtosis:                      34.625   Cond. No.                     5.50e+16
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.8e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [59]:
# re-calculate MAE
from sklearn.metrics import mean_absolute_error as mae
ypred = model.predict()
MAE = mae(Y, ypred)
MAE

10.842139061376743